In [1]:
import cv2
import mediapipe as mp
import numpy as np
import torch
import torch.nn as nn
from scipy.spatial import distance

objc[5544]: Class CaptureDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15c99a6b8) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x127e30860). One of the two will be used. Which one is undefined.
objc[5544]: Class CVWindow is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15c99a708) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x116a00a68). One of the two will be used. Which one is undefined.
objc[5544]: Class CVView is implemented in both /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15c99a730) and /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_h

In [2]:
# MPS 사용 가능 여부 확인
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS")
else:
    device = torch.device("cpu")
    print("CPU")

MPS


In [3]:

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 랜드마크 인덱스 정의 (예: 코, 왼쪽 어깨, 오른쪽 어깨 등)
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]  # 총 11개 랜드마크

# GRU 모델 정의
class GRUModel(torch.nn.Module):
    def __init__(self, input_size=27):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size = 64
        self.num_layers = num_layers = 2
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,
                          dropout=0.5)
        self.fc = nn.Linear(hidden_size, 3)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# GRU 모델 로드
input_size = 27
gru_model = GRUModel(input_size=input_size)  
gru_model.load_state_dict(torch.load('/Users/kimdeok-hwi/deeplearning/project/Project_humanFall/GRU/best_GRU_model_2.pt', map_location=torch.device('cpu')))
gru_model.eval()

# 클래스 이름 정의
class_names = {0: 'Normal', 1: 'Fall', 2: 'Danger'}

# Threshold 값 정의
threshold_normal = 6.5   # 일반 상태로 간주되는 속도 임계값
threshold_danger = 10.5   # 위험 상태로 간주되는 속도 임계값

def calculate_head_upper_body_speed(keypoints, prev_keypoints):
    h = np.array([keypoints[0, 0], keypoints[0, 1]])   # 머리 좌표
    l = np.array([keypoints[11, 0], keypoints[11, 1]])  # 왼쪽 어깨 좌표
    r = np.array([keypoints[12, 0], keypoints[12, 1]])  # 오른쪽 어깨 좌표

    # 이전 프레임의 좌표가 없는 경우 속도는 0으로 설정
    if prev_keypoints is None:
        return 0.0

    prev_h = np.array([prev_keypoints[0, 0], prev_keypoints[0, 1]])
    prev_l = np.array([prev_keypoints[11, 0], prev_keypoints[11, 1]])
    prev_r = np.array([prev_keypoints[12, 0], prev_keypoints[12, 1]])

    # 현재 프레임과 이전 프레임의 상체 중심 계산
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3

    # 유클리드 거리 계산 (속도)
    speed = distance.euclidean(center_new, center_prev)
    return speed

def process_landmarks(landmarks): 
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택 
    return selected_landmarks[:, :2].flatten()   # (x,y) 좌표 반환

def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))
    
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    
    # 높이가 0일 경우 비율을 무한대로 설정
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')
    
    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 패딩 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    return (x1, y1), (x2, y2), bbox_width, bbox_height

# 낙상 감지 함수
def detect_fall(landmarks, prev_landmarks):
    speed = calculate_head_upper_body_speed(landmarks, prev_landmarks)
    
    processed_landmarks = process_landmarks(landmarks)

    # 바운딩 박스 계산 및 그리기 
    top_left_bbox, bottom_right_bbox, bbox_width, bbox_height = calculate_and_draw_bbox(frame, landmarks)
    
    # 바운딩 박스 비율 계산
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')

    # 속도 기반 클래스 결정
    if speed < threshold_normal:
        bbox_class = 0   # Normal 
    elif speed < threshold_danger:
        bbox_class = 2   # Danger 
    else:
        bbox_class = 1   # Fall 
    
    # 바운딩 박스 비율에 따른 클래스 조정
    if bbox_class == 0 and bbox_ratio < 0.5:
        bbox_class = 0   # Normal에서 Normal로 조정
    elif bbox_class == 0 and 0.5 <= bbox_ratio <= 0.7: 
        bbox_class = 2   # Normal에서 Danger로 조정
    elif bbox_class == 0 and bbox_ratio > 1: 
        bbox_class = 1   # Normal에서 Fall로 조정
        
    elif bbox_class == 2 and bbox_ratio < 0.5: 
        bbox_class = 0
    elif bbox_class == 2 and 0.5 <= bbox_ratio <= 0.7: 
        bbox_class = 2
    elif bbox_class == 2 and bbox_ratio > 1: 
        bbox_class = 1
        
    elif bbox_class == 1 and bbox_ratio < 0.5: 
        bbox_class = 0
    elif bbox_class == 1 and 0.5 <= bbox_ratio <= 0.7: 
        bbox_class = 2
    elif bbox_class == 1 and bbox_ratio > 1: 
        bbox_class = 1

    return bbox_class

# 웹캠 열기
cap = cv2.VideoCapture(0)

prev_landmarks = None

# 비디오 속성 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 24

# 출력 비디오 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path = f'try_fall_detection_by_webcam.mp4' 
actual_fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

prev_landmarks = None

# 프레임 처리 루프 
while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_pose = pose.process(rgb_frame)

    if results_pose.pose_landmarks:
        landmarks = np.array([[lm.x * frame.shape[1], lm.y * frame.shape[0], lm.z] for lm in results_pose.pose_landmarks.landmark])
       
        if prev_landmarks is not None: 
            label = detect_fall(landmarks, prev_landmarks)  
            print(f"Predicted Class: {label}")  
        else: 
            label = None 

        prev_landmarks = landmarks 

        # 바운딩 박스와 라벨 그리기 
        top_left_bbox, bottom_right_bbox, _, _ = calculate_and_draw_bbox(frame, landmarks)  
        color = (0, 255, 0) if label == 0 else ((0, 255, 255) if label == 2 else (0, 0, 255)) 
        cv2.rectangle(frame, top_left_bbox, bottom_right_bbox, color, 2)
        class_name = class_names[label] if label is not None else 'Unknown'
        cv2.putText(frame, f'{class_name}', (top_left_bbox[0], top_left_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 3)

        # 랜드마크 표시 
        mp_drawing.draw_landmarks(frame, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # 프레임 저장
    out.write(frame)

    # 프레임 출력 
    cv2.imshow('Fall Detection', frame) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
         break

cap.release()
out.release()
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-11-23 18:23:22.433 Python[5544:146901] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-11-23 18:23:24.682 Python[5544:146901] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-23 18:23:24.682 Python[5544:146901] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 0
Predicted Class: 2
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 2
Predicted Class: 0
Predicted Class: 0
Predicted Class: 2
Predicted Class: 2
Predicted Class: 2
Predicted Class: 0
Predicted Class: 1
Predicted Class: 0
Predicted Class: 2
Predicted Class: 0
Predicted Class: 2
Predicted Class: 2
Predicted Class: 2
Predicted Class: 2
Predicted Class: 2
Predicted Class: 2
Predicted Class: 2
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 1
Predicted Class: 2
Predicted Class: 0
Predicted Class: 0
Predicted Class: 0
Predicted Class: 2
Predicted Class: 2
Predicted Cl

-1